In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt  
import seaborn as sns 
from pandas.plotting import scatter_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('healthcare-dataset-stroke-data.csv')
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [5]:
# Remove records with missing values
df = df[df['bmi'].notna()]

In [6]:
columns_list = df.columns

In [8]:
X = df.drop(columns_list[-1], axis = 1).values
y = df['stroke'].values

In [9]:
X.shape

(4909, 11)

In [10]:
y.shape

(4909,)

In [11]:
# Encode Data
for i in range(len(X)):
  X[i][X[i] == 'Male'] = 1
  X[i][X[i] == 'Female'] = 0
  X[i][X[i] == 'Other'] = 2

  X[i][X[i] == 'Yes'] = 1
  X[i][X[i] == 'No'] = 0

  X[i][X[i] == 'Govt_job'] = 2
  X[i][X[i] == 'Never_worked'] = 0
  X[i][X[i] == 'Private'] = 1
  X[i][X[i] == 'Self-employed'] = 3
  X[i][X[i] == 'children'] = 4

  X[i][X[i] == 'Rural'] = 1
  X[i][X[i] == 'Urban'] = 0

  X[i][X[i] == 'Unknown'] = 3
  X[i][X[i] == 'never smoked'] = 0
  X[i][X[i] == 'formerly smoked'] = 1
  X[i][X[i] == 'smokes'] = 2

In [12]:
# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

In [21]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
sc_y = StandardScaler()

X_std = sc_x.fit_transform(X_train)
X_std_test = sc_x.fit_transform(X_test)

y_std = y_train
y_std_test = y_test

# y_std = sc_y.fit_transform(y_train.reshape(-1,1)).flatten()
# y_std_test = sc_y.fit_transform(y_test.reshape(-1,1)).flatten()

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import sparse_categorical_crossentropy

In [14]:
model = Sequential([
    Dense(units = 11, input_shape = (11,), activation = 'relu'),
    Dense(units = 2, activation = 'softmax')
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 11)                132       
                                                                 
 dense_1 (Dense)             (None, 2)                 24        
                                                                 
Total params: 156
Trainable params: 156
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(optimizer = Adam(learning_rate = 0.0001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [22]:
model.fit(x = X_std, y = y_std, validation_split = 0.1, batch_size = 10, epochs = 10)

Epoch 1/10
354/354 [==============================] - 5s 4ms/step - loss: 1.0804 - accuracy: 0.2963 - val_loss: 1.0137 - val_accuracy: 0.3155
Epoch 2/10
354/354 [==============================] - 1s 3ms/step - loss: 0.8464 - accuracy: 0.4185 - val_loss: 0.8203 - val_accuracy: 0.4402
Epoch 3/10
354/354 [==============================] - 1s 3ms/step - loss: 0.6865 - accuracy: 0.5549 - val_loss: 0.6836 - val_accuracy: 0.5471
Epoch 4/10
354/354 [==============================] - 1s 3ms/step - loss: 0.5740 - accuracy: 0.6720 - val_loss: 0.5841 - val_accuracy: 0.6616
Epoch 5/10
354/354 [==============================] - 1s 4ms/step - loss: 0.4904 - accuracy: 0.7782 - val_loss: 0.5082 - val_accuracy: 0.7812
Epoch 6/10
354/354 [==============================] - 1s 3ms/step - loss: 0.4260 - accuracy: 0.8602 - val_loss: 0.4484 - val_accuracy: 0.8779
Epoch 7/10
354/354 [==============================] - 1s 3ms/step - loss: 0.3754 - accuracy: 0.9216 - val_loss: 0.4011 - val_accuracy: 0.9288
Epoch 

In [23]:
model.evaluate(X_std_test, y_std_test)

31/31 [==============================] - 0s 3ms/step - loss: 0.2971 - accuracy: 0.9450


[0.2970512807369232, 0.9450101852416992]